# **Import Library**

In [117]:
!pip install nltk
!pip install emoji
!pip install swifter
!pip install Sastrawi

import swifter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import re
import emoji
import string
import pandas as pd
import numpy as np

import nltk, re, pprint
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
nltk.download('gutenberg')
nltk.download('nps_chat')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('indonesian'))
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
from textblob import TextBlob
from nltk.stem import PorterStemmer
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.metrics import classification_report

from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

#Word2Vec
!pip install gensim
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Dataset**

In [118]:
#Access permission dataset melalui google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [119]:
# #Pemanggilan dataset
data = pd.read_csv("/content/drive/MyDrive/skripsi/data/twitterIKN.csv")
df = pd.DataFrame(data)
df.head()

,author id,date,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet,target
0,1.484686e+18,2022-01-27,2022-01-27T07:27:53.000Z,NaN,1.486602e+18,in,0,0,0,0,Twitter Web App,#AyoTolakUUIKN \n\nProyek IKN membuka peluang ...,0
1,1.464432e+18,2022-01-27,2022-01-27T07:27:54.000Z,NaN,1.486602e+18,in,0,0,0,0,Twitter for Android,Tolak UU IKN karena berpotensi merusak lingkun...,0
2,8.852835e+17,2022-01-27,2022-01-27T07:28:03.000Z,NaN,1.486602e+18,in,0,0,0,0,Twitter Web App,UU IKN hanya akan merugikan rakyat dengan huta...,0
3,1.384007e+18,2022-01-27,2022-01-27T07:28:04.000Z,NaN,1.486602e+18,in,1,0,0,0,Twitter Web App,Jika UU ini diterapkan yang terjadi adalah mas...,0
4,1.450712e+18,2022-01-27,2022-01-27T07:28:05.000Z,NaN,1.486602e+18,in,0,0,0,0,Twitter for Android,UU IKN Syarat kepentingan oligarki. Tolak n ba...,0


# **Data cleansing**

In [120]:
# Hapus kolom yang tidak diinginkan pada dataset
df = df.drop(['author id', 'date', 'created_at', 'geo', 'id', 'lang', 'like_count', 'quote_count', 'reply_count',
                                      'retweet_count', 'source'], axis=1)

df.head()

,tweet,target
0,#AyoTolakUUIKN \n\nProyek IKN membuka peluang ...,0
1,Tolak UU IKN karena berpotensi merusak lingkun...,0
2,UU IKN hanya akan merugikan rakyat dengan huta...,0
3,Jika UU ini diterapkan yang terjadi adalah mas...,0
4,UU IKN Syarat kepentingan oligarki. Tolak n ba...,0


# **Pre-processing**

In [121]:
import re
import emoji
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def process_hashtag(input_text: str) -> str:
    return re.sub(
        r'#[a-zA-Z]\S*',
        lambda m: ' '.join(re.findall('[A-Z][^A-Z]*|[a-z][^A-Z]*', m.group().lstrip('#'))), input_text,
    )

def remove_hashtag(tweet):
    tweet = process_hashtag(tweet)
    return tweet

def case_folding(text):
    text = text.lower()
    return text

def process_emoji(text):
    return emoji.replace_emoji(text, replace='')

def process_tweet(tweet):
    # Penghapusan URL
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags = re.MULTILINE)
    # Penghapusan user
    tweet = re.sub(r"\@\w+", '', tweet)
    # Penghapusan hashtag
    tweet = remove_hashtag(tweet)
    # Penghapusan karakter 1-length
    tweet = re.sub(r"\b\w{1}\b", '', tweet)
    # Penghapusan emoji
    tweet = process_emoji(tweet)
    # Penghapusan tanda baca (punctuations)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Penghapusan white spaces multi menjadi tunggal
    tweet = re.sub(r"[\s]+", ' ', tweet)
    # Penghapusan abbreviations
    tweet = re.sub(r"\b([b-df-hj-np-tv-z]){2,}\b|\b(amp)\b", '', tweet)
    # Trim
    tweet = tweet.strip('\'"')
    # Penghapusan duplicate consecutive words
    tweet = re.sub(r"\b(\w+)\s\1\b", r"\1", tweet)
    # Make lowercase
    tweet = tweet.lower()
    # Penghapusan stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [stemmer.stem(w) for w in tweet_tokens if w not in stop_words]
    return " ".join(filtered_words)

def preprocess(df):
    df['remove_hashtag'] = df['tweet'].apply(lambda x: remove_hashtag(x))
    df['casefolding'] = df['remove_hashtag'].apply(lambda x: case_folding(x))
    df['clean'] = df['casefolding'].apply(process_tweet)
    return df


In [122]:
# df = preprocess(df)
# df.head()

In [123]:
# remove_hastag('#ayotolakuuikn')

In [124]:
# remove_hastag('#AyoTolakUUIkn')

# **Pickle**

In [125]:
def write_pickle_file(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

In [126]:
def read_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

In [127]:
# df.to_excel("/content/drive/MyDrive/skripsi/data/final.xlsx")

In [128]:
df = pd.read_excel('/content/drive/MyDrive/skripsi/data/final.xlsx')

In [129]:
df.tail()

,Unnamed: 0.1,Unnamed: 0,tweet,target,remove_hashtag,casefolding,clean
1722,1722,1722,♻Visi Pembangunan IKN♻\n\nIKN akan dibangun de...,1,♻Visi Pembangunan IKN♻\n\nIKN akan dibangun de...,♻visi pembangunan ikn♻\n\nikn akan dibangun de...,visi bangun ikn bangun visi kota dunia worl...
1723,1723,1723,✍️ Menteri @Suharso_M: Berada di Tengah Indone...,1,✍️ Menteri @Suharso_M: Berada di Tengah Indone...,✍️ menteri @suharso_m: berada di tengah indone...,menteri indonesia ikn harap pusat gravitasi ek...
1724,1724,1724,✍️ Visi Pembangunan IKN\n\nIKN akan dibangun d...,1,✍️ Visi Pembangunan IKN\n\nIKN akan dibangun d...,✍️ visi pembangunan ikn\n\nikn akan dibangun d...,visi bangun ikn bangun visi kota dunia worl...
1725,1725,1725,"Berada di tengah Indonesia, IKN diharapkan men...",1,"Berada di tengah Indonesia, IKN diharapkan men...","berada di tengah indonesia, ikn diharapkan men...",indonesia ikn harap pusat gravitasi ekonomi ba...
1726,1726,1726,Pembangunan IKN usung konsep future smart fore...,1,Pembangunan IKN usung konsep future smart fore...,pembangunan ikn usung konsep future smart fore...,bangun ikn usung konsep future smart forest ci...


# **Split Data Training and Testing**

In [130]:
from sklearn.model_selection import train_test_split

def split_data(X, y, test_size):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return X_train, X_test, y_train, y_test

# **Evaluation Performance**

In [131]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report, confusion_matrix


def model_gnb(X_train, y_train, X_test_feature):
    gnb = GaussianNB()
    gnb.fit(X_train, y_train) #ini model
    pred = gnb.predict(X_test_feature)
    return [{'pred': pred, 'model':gnb }]


In [132]:
def evaluation(pred, y_test):
    cm = confusion_matrix(y_test, pred)
    tn, fp, fn, tp = cm.ravel()

    # Menghitung evaluasi (precision, recall, F1-score)
    accuracy = (tp+tn)/(tp+fn+fp+tn)
    precision_pos = tp/(tp+fp)
    precision_neg = tn/(tn+fn)
    precision = (precision_pos + precision_neg) / 2

    recall_pos = tp/(tp+fn)
    recall_neg = tn/(tn+fp)
    recall = (recall_pos + recall_neg) / 2

    f1_pos = 2 * (precision_pos * recall_pos) / (precision_pos + recall_pos)
    f1_neg = 2 * (precision_neg * recall_neg) / (precision_neg + recall_neg)
    f1 = f1_avg = (f1_pos + f1_neg) / 2

    # Menyimpan hasil evaluasi dalam sebuah list
    evaluation = [accuracy, precision, recall, f1]

    results = []
    positive_count = 0
    negative_count = 0
    for prediction in pred:
        if prediction == 1:
            positive_count += 1
        elif prediction == 0:
            negative_count += 1
    results.append(positive_count)
    results.append(negative_count)
    #param value utk di panggil ke pickled
    print({'evaluation':evaluation})
    return [{'confusion':cm, 'result': results, 'evaluation':evaluation}]

# **Word2Vec**

In [133]:
from gensim.models import Word2Vec
import numpy as np
from tqdm import tqdm

def train_word2vec_model(sentences):
    model = Word2Vec(
        window=8,
        sg=1,
        vector_size=300
    )

    model.build_vocab(sentences)
    model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

    w2v_words = set(model.wv.index_to_key)

    sent_vectors = []
    for sent in tqdm(sentences):
        sent_vec = np.zeros(300)
        cnt_words = 0
        for word in sent:
            if word in w2v_words:
                vec = model.wv[word]
                sent_vec += vec
                cnt_words += 1
        if cnt_words != 0:
            sent_vec /= cnt_words
        sent_vectors.append(sent_vec)

    return model, sent_vectors


In [134]:
list_sentence = [sentence.split() for sentence in df['clean']]
w2v_model, sent_vectors = train_word2vec_model(list_sentence)

100%|██████████| 1727/1727 [00:01<00:00, 983.92it/s]


In [135]:
X=np.array(sent_vectors)
y = df['target'].values
print("x shape=",X.shape,"  y shape=",y.shape)

x shape= (1727, 300)   y shape= (1727,)


# **30% Testing Result**




In [136]:
X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.30)

In [137]:
w2v_model1 = model_gnb(X_train, y_train, X_test)
w2v_evaluate1 = evaluation(w2v_model1[0]['pred'], y_test)

{'evaluation': [0.8940269749518305, 0.9005783925820774, 0.8939040689040689, 0.8935702168117672]}


# **20% Testing Result**

In [138]:
X_train2, X_test2, y_train2, y_test2 = split_data(X, y, test_size=0.20)

In [139]:
w2v_model2 = model_gnb(X_train2, y_train2, X_test2)
w2v_evaluate2 = evaluation(w2v_model2[0]['pred'], y_test2)

{'evaluation': [0.8670520231213873, 0.8879903395784543, 0.8569552922165364, 0.8620319001386962]}


# **Save Word2Vec Model**

In [140]:
import pickle
file_path = '/content/drive/MyDrive/skripsi/data/model/w2v.pkl'
w2v_model = w2v_model1[0]['model']
write_pickle_file(w2v_model, file_path)

In [141]:
result = read_pickle_file('/content/drive/MyDrive/skripsi/data/model/w2v.pkl')
result.predict(X_test)

array([1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,

# **Load Model Word2Vec**

In [142]:
result = read_pickle_file('/content/drive/MyDrive/skripsi/data/model/w2v.pkl')
print(result)

GaussianNB()


# **Train other data with existing model**

In [143]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

def plot_learning_curves(estimator, X, y):
    train_sizes, train_scores, valid_scores = learning_curve(estimator, X, y, cv=5, scoring='accuracy', train_sizes=np.linspace(0.1, 1.0, 10))

    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    valid_scores_std = np.std(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Training score')
    plt.plot(train_sizes, valid_scores_mean, 'o-', color='g', label='Cross-validation score')

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color='r')
    plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std, valid_scores_mean + valid_scores_std, alpha=0.1, color='g')

    plt.xlabel('Training examples')
    plt.ylabel('Score')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

In [144]:
for i in range(5):
  list_sentence = [sentence.split() for sentence in df['clean']]
  w2v_model, sent_vectors = train_word2vec_model(list_sentence)

  X=np.array(sent_vectors)
  y = df['target'].values

  X_train2, X_test2, y_train2, y_test2 = split_data(X, y, test_size=0.30)
  result.fit(X_train2, y_train2)
  pred2 = result.predict(X_test2)
  evaluations = evaluation(pred2, y_test2)

100%|██████████| 1727/1727 [00:01<00:00, 893.49it/s]


{'evaluation': [0.8882466281310212, 0.8945834581959844, 0.8924664679582712, 0.8882130124777183]}


100%|██████████| 1727/1727 [00:01<00:00, 1399.39it/s]


{'evaluation': [0.8921001926782274, 0.8975569933164464, 0.8884639470388263, 0.8906875282082142]}


100%|██████████| 1727/1727 [00:03<00:00, 490.87it/s]


{'evaluation': [0.8766859344894027, 0.8854658782651101, 0.8756757961026616, 0.8757518628243111]}


100%|██████████| 1727/1727 [00:01<00:00, 1381.45it/s]


{'evaluation': [0.8998073217726397, 0.907721465462563, 0.9010472370766489, 0.8994935200357514]}


100%|██████████| 1727/1727 [00:02<00:00, 779.53it/s]


{'evaluation': [0.884393063583815, 0.8944398944398945, 0.8864453624179025, 0.8839791356184798]}


# **TF-IDF**

In [174]:
y = df['target'].values
X = df[['clean']]

In [175]:
def transform_to_tfidf(X_train, X_test):
    vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 4))
    vectorizer.fit(X_train)
    X_train_tfidf = vectorizer.transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    X_train_tfidf = X_train_tfidf.toarray()
    X_test_tfidf = X_test_tfidf.toarray()
    return X_train_tfidf, X_test_tfidf

# **30% Testing Resultt**

In [176]:
X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.30)
X_train_tfidf, X_test_tfidf = transform_to_tfidf(X_train['clean'], X_test['clean'])
# tf_idf_evaluate = model_gnb(X_train_tfidf, y_train, X_test_tfidf, y_test)

In [177]:
tf_idf_model1 = model_gnb(X_train_tfidf, y_train, X_test_tfidf)
tf_idf_evaluate1 = evaluation(tf_idf_model1[0]['pred'], y_test)

{'evaluation': [0.9113680154142582, 0.910513643659711, 0.9130057717036992, 0.9110904215700879]}


# **20% Testing Result**

In [180]:
X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.20)
X_train_tfidf, X_test_tfidf = transform_to_tfidf(X_train['clean'], X_test['clean'])

In [181]:
tf_idf_model2 = model_gnb(X_train_tfidf, y_train, X_test_tfidf)
tf_idf_evaluate2 = evaluation(tf_idf_model2[0]['pred'], y_test)

{'evaluation': [0.9017341040462428, 0.9025047131699434, 0.9005628517823641, 0.9012590229981534]}


# **Save TF-IDF Model**

In [182]:
import pickle
file_path = '/content/drive/MyDrive/skripsi/data/model/tf-idf.pkl'

tf_idf_model = tf_idf_model1[0]['model']
write_pickle_file(tf_idf_model, file_path)

# **Load TF-IDF Model**

In [152]:
result = read_pickle_file('/content/drive/MyDrive/skripsi/data/model/tf-idf.pkl')
print(result)

GaussianNB()


# **Train other data with existing model**

In [153]:
tf_idf_means = 0
for i in range(5):
  y = df['target'].values
  X = df[['clean']]
  X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.30)
  X_train_tfidf, X_test_tfidf = transform_to_tfidf(X_train['clean'], X_test['clean'])

  result.fit(X_train_tfidf, y_train)
  pred = result.predict(X_test_tfidf)
  evaluations = evaluation(pred, y_test)

{'evaluation': [0.8882466281310212, 0.8893504869114626, 0.8882981882981883, 0.8881764688400844]}
{'evaluation': [0.884393063583815, 0.8854864647547575, 0.8844446094446095, 0.8843204850069839]}
{'evaluation': [0.8998073217726397, 0.8993896100038613, 0.9029442613125562, 0.8995354239256677]}
{'evaluation': [0.8998073217726397, 0.901842972729997, 0.8986888267824226, 0.8994006083378063]}
{'evaluation': [0.8940269749518305, 0.8933903345724907, 0.8944578636607622, 0.8937603513407895]}


# **Flask Config**

In [154]:
!pip install flask-ngrok
!pip install flask-login
!pip install flask-sqlalchemy
!pip install pyngrok

In [155]:
# !ngrok authtoken 2gscOxZKt6SseWDnfvegxZaJYqK_3uqhZJA4cTSNVewajBKrS
!ngrok authtoken 2hv6s0Tcxq00qIsmFCtr3hHVSPw_3pktmegp5jRboFSdfjg5p

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [156]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

file_path1 = '/content/drive/MyDrive/skripsi/data/model/tf-idf.pkl'
file_path2 = '/content/drive/MyDrive/skripsi/data/model/w2v.pkl'

def read_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

def evaluations(pred, y_test):
    cm = confusion_matrix(y_test, pred)
    tn, fp, fn, tp = cm.ravel()

    # Menghitung evaluasi (precision, recall, F1-score)
    accuracy = (tp+tn)/(tp+fn+fp+tn)
    precision_pos = tp/(tp+fp)
    precision_neg = tn/(tn+fn)
    precision = (precision_pos + precision_neg) / 2

    recall_pos = tp/(tp+fn)
    recall_neg = tn/(tn+fp)
    recall = (recall_pos + recall_neg) / 2

    f1_pos = 2 * (precision_pos * recall_pos) / (precision_pos + recall_pos)
    f1_neg = 2 * (precision_neg * recall_neg) / (precision_neg + recall_neg)
    f1 = f1_avg = (f1_pos + f1_neg) / 2

    # Menyimpan hasil evaluasi dalam sebuah list
    evaluation = [accuracy, precision_pos, precision_neg, recall_pos, recall_neg, f1_pos, f1_neg]

    results = []
    positive_count = 0
    negative_count = 0
    for prediction in pred:
        if prediction == 1:
            positive_count += 1
        elif prediction == 0:
            negative_count += 1
    results.append(positive_count)
    results.append(negative_count)
    #param value utk di panggil ke pickled
    print({'confusion':cm, 'result': results, 'evaluation':evaluation})
    return [{'confusion':cm, 'result': results, 'evaluation':evaluation}]

app = Flask(__name__, template_folder='/content/drive/MyDrive/skripsi/html/')
run_with_ngrok(app)

tf_idf_model = read_pickle_file(file_path1)
w2v_models = read_pickle_file(file_path2)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/show-data', methods=['POST'])
def show_data():
  global new_df
  file = request.files['file']
  if file.filename == '':
      return "No selected file"
  if file.filename.endswith('.csv'):
      new_df = pd.read_csv(file)
  elif file.filename.endswith('.xls') or file.filename.endswith('.xlsx'):
      new_df = pd.read_excel(file)
  else:
      return "Invalid file type"

  new_df = preprocess(new_df)

  return render_template('index.html', data=new_df.to_dict(orient='records'))

@app.route('/pie.html', methods=['GET', 'POST'])
def getprediction():
  selected_model = request.args.get('model')

  if selected_model == 'TF-IDF':
      y = new_df['target'].values
      X = new_df[['clean']]
      X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.30)
      X_train_tfidf, X_test_tfidf = transform_to_tfidf(X_train['clean'], X_test['clean'])

      tf_idf_model.fit(X_train_tfidf, y_train)
      pred = tf_idf_model.predict(X_test_tfidf)
      tf_idf_evaluate = evaluations(pred, y_test)

      prediction_data = tf_idf_evaluate[0]['result']
      confusion = tf_idf_evaluate[0]['confusion']
      evaluation = tf_idf_evaluate[0]['evaluation']


  elif selected_model == 'Word2vec':
      list_sentence = [sentence.split() for sentence in df['clean']]
      w2v_model, sent_vectors = train_word2vec_model(list_sentence)

      X=np.array(sent_vectors)
      y = df['target'].values

      X_train2, X_test2, y_train2, y_test2 = split_data(X, y, test_size=0.30)
      w2v_models.fit(X_train2, y_train2)
      pred2 = w2v_models.predict(X_test2)
      w2v_evaluate = evaluations(pred2, y_test2)

      prediction_data = w2v_evaluate[0]['result']
      confusion = w2v_evaluate[0]['confusion']
      evaluation = w2v_evaluate[0]['evaluation']

  return render_template('pie.html', confusion = confusion, evaluation=evaluation, pie_data=prediction_data)

if __name__ == '__main__':
    http_tunnel = ngrok.connect(5000)
    print("Public URL:", http_tunnel.public_url)
    app.run()


Public URL: https://1bc2-34-19-74-120.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1bc2-34-19-74-120.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/Jul/2024 15:10:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2024 15:10:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2024 15:10:21] "POST /show-data HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jul/2024 15:10:33] "GET /pie.html?model=TF-IDF HTTP/1.1" 200 -


{'confusion': array([[239,  20],
       [ 40, 220]]), 'result': [240, 279], 'evaluation': [0.884393063583815, 0.9166666666666666, 0.8566308243727598, 0.8461538461538461, 0.9227799227799228, 0.8799999999999999, 0.8884758364312267]}


100%|██████████| 1727/1727 [00:00<00:00, 24333.22it/s]
INFO:werkzeug:127.0.0.1 - - [29/Jul/2024 15:10:44] "GET /pie.html?model=Word2vec HTTP/1.1" 200 -


{'confusion': array([[202,  58],
       [ 16, 243]]), 'result': [301, 218], 'evaluation': [0.8574181117533719, 0.8073089700996677, 0.926605504587156, 0.9382239382239382, 0.7769230769230769, 0.8678571428571429, 0.8451882845188285]}
